In [1]:
import requests
import base64


In [2]:
CLIENT_ID = '306e8a22cf0549a49a4d7c86ee9008a3'
CLIENT_SECRET = 'f0591016a12148f4a9b4cd374d341212'

# Base64 mã hóa client ID và client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

In [3]:
# Yêu cầu mã thông báo truy cập
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Mã thông báo truy cập đã được lấy thành công.")
else:
    print("Lỗi lấy mã thông báo truy cập.")
    exit()

Mã thông báo truy cập đã được lấy thành công.


In [4]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_trending_playlist_data(playlist_id, access_token):
    # Thiết lập Spotipy bằng mã thông báo truy cập
    sp = spotipy.Spotify(auth=access_token)

    # Nhận các bản nhạc từ danh sách phát
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    # Trích xuất thông tin liên quan và lưu trữ trong danh sách từ điển
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # Nhận các tính năng âm thanh cho bản nhạc
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # Nhận ngày phát hành album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

       # Nhận được sự phổ biến của bài hát
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # Thêm thông tin bản nhạc bổ sung vào dữ liệu bản nhạc
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            
        }

        music_data.append(track_data)

    
    df = pd.DataFrame(music_data)

    return df

In [5]:
playlist_id = '0s26Op11drslSFu5WgWVWJ'

# Gọi hàm để lấy dữ liệu nhạc từ danh sách phát và lưu trữ nó trong một DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

print(music_df)

                   Track Name                Artists  \
0             Khúc Vương Tình      Cần Vinh, Lee Ken   
1             ĐỢI (Prod. RIO)                   52Hz   
2                    Tò Te Tí      Wren Evans, itsnk   
3                     Em Xinh          MONO, Onionn.   
4      Anh Là Ngoại Lệ Của Em              Phương Ly   
..                        ...                    ...   
95                   Chìm Sâu    RPT MCK, Trung Trần   
96  bao tiền một mớ bình yên?  14 Casper, Bon Nghiêm   
97                      Em Là          MONO, Onionn.   
98         Chờ Đợi Có Đáng Sợ                 Andiez   
99          Ngồi Nhìn Em Khóc                    Sáo   

                    Album Name                Album ID  \
0              Khúc Vương Tình  2IqbsCGagR8OSwuKhgFZEe   
1              ĐỢI (Prod. RIO)  1lUZ3071nJ3asd2DBQy6dm   
2   LOI CHOI: The Neo Pop Punk  1c4nTHI2hreFeF5P37wf4f   
3                          ĐẸP  0Y4fLNVEo2CKhaHEb0CjJ3   
4       Anh Là Ngoại Lệ Của Em  7kHOy

In [6]:
print(music_df.isnull().sum())

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

data = music_df

In [8]:
# Chức năng tính điểm phổ biến có trọng số dựa trên ngày phát hành
def calculate_weighted_popularity(release_date):
    # Chuyển đổi ngày phát hành thành đối tượng datetime
    release_date = datetime.strptime(release_date, '%Y-%m-%d')

    # Tính khoảng thời gian giữa ngày phát hành và ngày hôm nay
    time_span = datetime.now() - release_date

    # Tính điểm phổ biến có trọng số dựa trên khoảng thời gian (ví dụ: các bản phát hành gần đây hơn có trọng số cao hơn)
    weight = 1 / (time_span.days + 1)
    return weight

In [9]:
# Chuẩn hóa các tính năng âm nhạc bằng cách sử dụng tỷ lệ Min-Max
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key', 
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [10]:
# chức năng nhận đề xuất dựa trên nội dung dựa trên các tính năng âm nhạc
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Lấy chỉ mục của bài hát đầu vào trong DataFrame nhạc
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    # Tính điểm tương tự dựa trên đặc điểm âm nhạc (độ tương tự cosine)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    # Lấy chỉ số của các bài hát giống nhau nhất
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Lấy tên các bài hát giống nhau nhất dựa trên tính năng lọc dựa trên nội dung
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

In [11]:

import pandas as pd

def hybrid_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Nhận đề xuất dựa trên nội dung
    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    # Lấy hàng cho bài hát đầu vào
    input_song_row = music_df[music_df['Track Name'] == input_song_name]

    # Tính điểm phổ biến có trọng số
    weighted_popularity_score = input_song_row['Popularity'].iloc[0] * calculate_weighted_popularity(input_song_row['Release Date'].iloc[0])

   # Tạo DataFrame mới cho bài hát đầu vào với mức độ phổ biến có trọng số
    input_song_df = input_song_row.copy()
    input_song_df['Popularity'] = weighted_popularity_score

    # Kết hợp các đề xuất dựa trên nội dung và mức độ phổ biến
    hybrid_recommendations_df = pd.concat([content_based_rec, input_song_df])

    # Sắp xếp các đề xuất kết hợp dựa trên điểm phổ biến có trọng số
    hybrid_recommendations_df = hybrid_recommendations_df.sort_values(by='Popularity', ascending=False)

    # Xóa bài hát đầu vào khỏi đề xuất
    hybrid_recommendations_df = hybrid_recommendations_df[hybrid_recommendations_df['Track Name'] != input_song_name]

    return hybrid_recommendations_df

In [12]:
def get_external_urls(input_song_name, recommended_songs):
    external_urls = {}

    # Truy xuất URL bên ngoài của bài hát đầu vào
    input_song_row = music_df[music_df['Track Name'] == input_song_name]
    if not input_song_row.empty:
        input_song_url = input_song_row['External URLs'].iloc[0]
        external_urls[input_song_name] = input_song_url
    else:
        print(f"URL not found for '{input_song_name}'.")
        external_urls[input_song_name] = None

    # Truy xuất URL bên ngoài của các bài hát được đề xuất
    for song_name in recommended_songs:
        song_row = music_df[music_df['Track Name'] == song_name]
        if not song_row.empty:
            external_url = song_row['External URLs'].iloc[0]
            external_urls[song_name] = external_url
        else:
            print(f"URL not found for '{song_name}'.")
            external_urls[song_name] = None

    return external_urls


In [13]:
data = music_df 
print(data)

                   Track Name                Artists  \
0             Khúc Vương Tình      Cần Vinh, Lee Ken   
1             ĐỢI (Prod. RIO)                   52Hz   
2                    Tò Te Tí      Wren Evans, itsnk   
3                     Em Xinh          MONO, Onionn.   
4      Anh Là Ngoại Lệ Của Em              Phương Ly   
..                        ...                    ...   
95                   Chìm Sâu    RPT MCK, Trung Trần   
96  bao tiền một mớ bình yên?  14 Casper, Bon Nghiêm   
97                      Em Là          MONO, Onionn.   
98         Chờ Đợi Có Đáng Sợ                 Andiez   
99          Ngồi Nhìn Em Khóc                    Sáo   

                    Album Name                Album ID  \
0              Khúc Vương Tình  2IqbsCGagR8OSwuKhgFZEe   
1              ĐỢI (Prod. RIO)  1lUZ3071nJ3asd2DBQy6dm   
2   LOI CHOI: The Neo Pop Punk  1c4nTHI2hreFeF5P37wf4f   
3                          ĐẸP  0Y4fLNVEo2CKhaHEb0CjJ3   
4       Anh Là Ngoại Lệ Của Em  7kHOy

In [14]:
input_song_name = "Đường Tôi Chở Em Về"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Các bài hát được đề xuất kết hợp cho '{input_song_name}':")
print(recommendations)

Các bài hát được đề xuất kết hợp cho 'Đường Tôi Chở Em Về':
                                Track Name                Artists  \
33                                Hư không                    Kha   
60                                nhắn nhủ               Ronboogz   
58  Không Yêu Xin Đừng Nói - Piano Version                   UMIE   
11                    ghệ iu dấu của em ơi  tlinh, 2pillz, WOKEUP   
78                  Anh Đã Quen Với Cô Đơn                 SOOBIN   

                                Album Name Release Date  Popularity Album ID  \
33                                    tình   2023-12-14        64.0      NaN   
60                                nhắn nhủ   2023-12-12        64.0      NaN   
58  Không Yêu Xin Đừng Nói (Piano Version)   2022-01-22        61.0      NaN   
11                                      ái   2023-08-16        55.0      NaN   
78                  Anh Đã Quen Với Cô Đơn   2017-03-14        54.0      NaN   

   Track ID  Duration (ms) Explicit External

In [15]:
external_urls = get_external_urls(input_song_name, recommendations['Track Name'])

# In các URL bên ngoài
print("URL dẫn đến bài hát:")
for song, url in external_urls.items():
    print(f"{song}: {url}")

URL dẫn đến bài hát:
Đường Tôi Chở Em Về: https://open.spotify.com/track/27F22WcGDyl4Z2KylVJkXL
Hư không: https://open.spotify.com/track/3A9A7mfVDVm3rrS9SMGhg5
nhắn nhủ: https://open.spotify.com/track/2uZmeuNAxSm7XfVIwLkhCx
Không Yêu Xin Đừng Nói - Piano Version: https://open.spotify.com/track/0vkExWiBJSrfWnlMC7x48b
ghệ iu dấu của em ơi: https://open.spotify.com/track/4YSryHqsD82Krc81l1EJD5
Anh Đã Quen Với Cô Đơn: https://open.spotify.com/track/39KWnPNEvbXt423CzjvBfA
